# Lesson 6-Task 1
## Build and Train a Simple ANN Model

In [1]:
# Imports and dataset load
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import joblib

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
os.listdir('/content/drive/MyDrive/Colab Notebooks')

['Untitled0.ipynb', 'SLIIT AIML']

In [14]:
csv_path = '/content/drive/MyDrive/Colab Notebooks/SLIIT AIML/task-1/Mobile_Price_Classification-220531-204702.csv'
df = pd.read_csv(csv_path)
print('Dataset shape:', df.shape)
print('Price range unique values:', df['price_range'].unique())

Dataset shape: (2000, 21)
Price range unique values: [1 0]


In [15]:
# Preprocessing and train/test split
# Convert to binary if dataset has >2 classes
if df['price_range'].nunique() > 2:
    df['price_range'] = (df['price_range'] > 0).astype(int)
    print('Converted to binary. New unique values:', df['price_range'].unique())

# Split features and target
X = df.drop(columns=['price_range']).values
y = df['price_range'].values

# Train-test split (75% train, 25% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


Train shape: (1500, 20) Test shape: (500, 20)


In [16]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [17]:
# Build ANN model (2 hidden layers: 8 and 4 neurons)
model = Sequential([
    Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209 (836.00 B)

 Trainable params: 209 (836.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4998 - loss: 0.7943 - val_accuracy: 0.5100 - val_loss: 0.7351
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5085 - loss: 0.7363 - val_accuracy: 0.4960 - val_loss: 0.7148
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5460 - loss: 0.6966 - val_accuracy: 0.5000 - val_loss: 0.7057
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5411 - loss: 0.6988 - val_accuracy: 0.5240 - val_loss: 0.6991
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5447 - loss: 0.6899 - val_accuracy: 0.5220 - val_loss: 0.6957
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5463 - loss: 0.6868 - val_accuracy: 0.5360 - val_loss: 0.6923
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5679 - loss: 0.6828 - val_accuracy: 0.5480 - val_loss: 0.6892
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6029 - loss: 0.6774 - val_accuracy: 0.5500 - 

In [20]:
# Evaluate and save
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}, Test accuracy: {acc:.4f}')

# Save weights and scaler
weights_file = 'mobile_price_weights.weights.h5'
model.save_weights(weights_file)
print('Saved model weights to', weights_file)

scaler_file = 'scaler.save'
joblib.dump(scaler, scaler_file)
print('Saved feature scaler to', scaler_file)

Test loss: 0.3676, Test accuracy: 0.8340
Saved model weights to mobile_price_weights.weights.h5
Saved feature scaler to scaler.save
